In [11]:
from ib_async import *
import requests, pytz
import pandas as pd
import numpy as np
from datetime import datetime, time, timedelta
import pandas_market_calendars as mcal
from zoneinfo import ZoneInfo
import yfinance as yf
from data_and_research import ac

In [12]:
def get_last_full_trading_day(current_datetime=None):
    # Create NYSE calendar
    nyse = mcal.get_calendar('NYSE')
    
    # Get NYSE timezone
    nyse_tz = pytz.timezone('America/New_York')
    
    # Use provided datetime or current time if none provided
    if current_datetime is None:
        current_datetime = datetime.now(pytz.timezone('Europe/Berlin'))
    elif current_datetime.tzinfo is None:
        current_datetime = pytz.timezone('Europe/Berlin').localize(current_datetime)
    
    # Convert to NYSE time
    nyse_time = current_datetime.astimezone(nyse_tz)
    
    # Get market schedule for the current day and the previous few days
    schedule = nyse.schedule(start_date=nyse_time.date() - timedelta(days=5), end_date=nyse_time.date())
    
    if not schedule.empty:
        # Get the last row of the schedule (should be today or the most recent trading day)
        last_day = schedule.iloc[-1]
        market_close = last_day['market_close'].tz_convert(nyse_tz)
        
        # If current time is before the market close of the last day in the schedule,
        # we need to look at the previous trading day
        if nyse_time < market_close:
            if len(schedule) > 1:
                return schedule.index[-2].date()
            else:
                # If there's only one day in the schedule, find the previous trading day
                previous_trading_days = nyse.valid_days(end_date=nyse_time.date() - timedelta(days=1), start_date=nyse_time.date() - timedelta(days=5))
                return previous_trading_days[-1].date()
        else:
            # If current time is after market close, the last day in the schedule is the last full trading day
            return last_day.name.date()
    else:
        # If there's no schedule for the recent days (e.g., long weekend), 
        # find the last trading day
        previous_trading_days = nyse.valid_days(end_date=nyse_time.date() - timedelta(days=1), start_date=nyse_time.date() - timedelta(days=5))
        return previous_trading_days[-1].date()

In [13]:
get_last_full_trading_day()

datetime.date(2024, 10, 4)

In [14]:
def get_current_or_next_trading_day(current_datetime=None):
    # Create NYSE calendar
    nyse = mcal.get_calendar('NYSE')
    
    # Get NYSE timezone
    nyse_tz = pytz.timezone('America/New_York')
    
    # Use provided datetime or current time if none provided
    if current_datetime is None:
        current_datetime = datetime.now(pytz.timezone('Europe/Berlin'))
    elif current_datetime.tzinfo is None:
        current_datetime = pytz.timezone('Europe/Berlin').localize(current_datetime)
    
    # Convert to NYSE time
    nyse_time = current_datetime.astimezone(nyse_tz)
    
    # Get market schedule for today and the next few days
    schedule = nyse.schedule(start_date=nyse_time.date(), end_date=nyse_time.date() + timedelta(days=10))
    
    if not schedule.empty:
        today_schedule = schedule.loc[schedule.index.date == nyse_time.date()]
        if not today_schedule.empty:
            market_open = today_schedule.iloc[0]['market_open'].tz_convert(nyse_tz)
            market_close = today_schedule.iloc[0]['market_close'].tz_convert(nyse_tz)
            
            # If the current time is before market close, return today
            if nyse_time <= market_close:
                return nyse_time.date()
    
    # If we're here, it means today is not a trading day or the market has closed
    # Find the next trading day
    next_trading_days = nyse.valid_days(start_date=nyse_time.date() + timedelta(days=1), end_date=nyse_time.date() + timedelta(days=10))
    return next_trading_days[0].date()

In [15]:
get_current_or_next_trading_day()

datetime.date(2024, 10, 7)

In [16]:
def get_earnings():
    # Get the last full trading day and the current or next trading day
    last_trading_day = get_last_full_trading_day()
    next_trading_day = get_current_or_next_trading_day()

    # Define the SQL query with the specific dates and conditions
    query = f"""
    WITH LatestEarnings AS (
        SELECT *,
            ROW_NUMBER() OVER (PARTITION BY act_symbol ORDER BY `date` DESC) AS rn
        FROM `earnings_calendar`
        WHERE `when` IS NOT NULL
    )
    SELECT *
    FROM LatestEarnings
    WHERE rn = 1
    AND (
        (`date` = '{last_trading_day}' AND `when` = 'After market close') OR
        (`date` = '{next_trading_day}' AND `when` = 'Before market open')
    )
    ORDER BY `act_symbol` ASC;
    """
    
    # URL encode the query
    encoded_query = requests.utils.quote(query)
    
    # Set the DoltHub API endpoint and parameters
    endpoint = f"https://www.dolthub.com/api/v1alpha1/post-no-preference/earnings/master?q={encoded_query}"
    
    # Make the API request
    response = requests.get(endpoint)
    response.raise_for_status()  # Check for errors

    # Check the content of the response
    data = response.json()

    # Handle the response and convert to a pandas DataFrame if successful
    if data.get('query_execution_status') == 'Success':
        df = pd.DataFrame(data['rows'])
        df = df.rename(columns={'act_symbol': 'symbol'})
        df = df[['symbol', 'date', 'when']]
        return df
    else:
        print(f"Query Error: {data.get('query_execution_message')}")

In [17]:
get_earnings()

KeyError: "None of [Index(['symbol', 'date', 'when'], dtype='object')] are in the [columns]"

In [22]:
def get_vol_data(symbols: list[str] = None, curated = True, include_yf = True):

    # Build the query to get the max date from the volatility_history table
    query_max_date = """
    SELECT MAX(`date`) AS max_date
    FROM `volatility_history`
    """
    encoded_query_max_date = requests.utils.quote(query_max_date)
    endpoint_max_date = f"https://www.dolthub.com/api/v1alpha1/post-no-preference/options/master?q={encoded_query_max_date}"
    response_max_date = requests.get(endpoint_max_date) # Make the request to DoltHub for the max date
    response_max_date.raise_for_status()  # Check for errors

    # Convert the response to a pandas DataFrame
    data_max_date = response_max_date.json()

    # Extract the maximum date from the response
    if data_max_date.get('query_execution_status') == 'Success' and data_max_date.get('rows'):
        max_date = data_max_date['rows'][0]['max_date']
        print("Looking up vol data for:", max_date)
    else:
        print(f"Query Error: {data_max_date.get('query_execution_message')}")
        max_date = None

    if not max_date:
        return None
    
    if not symbols:
        earnings_table = get_earnings()
        symbols = earnings_table['symbol'].tolist()

    # Build the query to get the vol data for the max date
    symbols_str = ', '.join(f"'{symbol}'" for symbol in symbols)
    
    query_volatility = f"""
    SELECT *
    FROM `volatility_history`
    WHERE `date` = '{max_date}'
    AND `act_symbol` IN ({symbols_str})
    ORDER BY `act_symbol` ASC;
    """
    encoded_query_volatility = requests.utils.quote(query_volatility)
    endpoint_volatility = f"https://www.dolthub.com/api/v1alpha1/post-no-preference/options/master?q={encoded_query_volatility}"
    response_volatility = requests.get(endpoint_volatility) # Make the request to DoltHub for volatility history
    response_volatility.raise_for_status()  # Check for errors

    # Convert the response to a pandas DataFrame
    data_volatility = response_volatility.json()

    # Check and convert the response to a pandas DataFrame
    if data_volatility.get('query_execution_status') == 'RowLimit':
        if data_volatility.get('rows'):
            df_volatility = pd.DataFrame(data_volatility['rows'])
        else:
            print("No data found for the maximum date.")
    else:
        df_volatility = pd.DataFrame(data_volatility['rows'])
    
    if not curated:
        return df_volatility
    
    df_vol = df_volatility[['act_symbol', 'date', 'hv_current', 'iv_current']].astype({'hv_current': 'float','iv_current': 'float'})
    df_vol['vol_premium'] = df_vol['iv_current']/df_vol['hv_current']

    return df_vol.sort_values(by='vol_premium', ascending=False)



In [23]:
vol_df = get_vol_data(symbols=['AAPL'])

Looking up vol data for: 2024-10-03


In [24]:
vol_df

,act_symbol,date,hv_current,iv_current,vol_premium
0,AAPL,2024-10-03,0.2191,0.2712,1.237791


In [25]:
def connect_to_IB(port=7497, clientid=0, symbol=None):
    util.startLoop()  # Needed in script mode
    ib = IB()
    try:
        ib.connect('127.0.0.1', port, clientId=clientid)
    except ConnectionError:
        ib = None  # Reset ib on failure
    return ib

In [26]:
ib = connect_to_IB()

In [27]:
ib.reqMarketDataType(2)


In [28]:
import asyncio

In [37]:
async def process_get_filtered_put_options(ib, symbol,min_dte=0, max_dte=60, unfiltered=False,min_safety_margin=0.05,max_safety_margin=0):
    
    # Get the stock contract
    stock = Stock(symbol, 'SMART', 'USD')
    await ib.qualifyContractsAsync(stock)
    
    # Get the stock price
    [ticker] = await ib.reqTickersAsync(stock)
    stock_price = ticker.marketPrice()
    
    if np.isnan(stock_price):
        print(f"Unable to get market price for {symbol}")
        return None, None
    
    # Calculate strike range
    lower_strike = stock_price * (1-min_safety_margin)  
    upper_strike = stock_price * (1+max_safety_margin) 
    
    # Get option chains
    chains = await ib.reqSecDefOptParamsAsync(stock.symbol, '', stock.secType, stock.conId)
    
    # Get the chain with exchange 'SMART'
    chain = next((c for c in chains if c.exchange == 'SMART'), None)
    if not chain:
        print(f"No SMART chain found for {symbol}")
        return None, None
    
    # Get current date
    today = datetime.now().date()
    
    # Filter expirations and strikes
    valid_expirations = [exp for exp in chain.expirations 
                         if (datetime.strptime(exp, '%Y%m%d').date() - today).days >= min_dte and (datetime.strptime(exp, '%Y%m%d').date() - today).days <= max_dte]
    valid_strikes = [strike for strike in chain.strikes 
                     if lower_strike <= strike <= upper_strike]
    
    # Create option contracts
    contracts = [Option(symbol, exp, strike, 'P', 'SMART') 
                 for exp in valid_expirations 
                 for strike in valid_strikes]
    
    # Qualify the contracts
    contracts = await ib.qualifyContractsAsync(*contracts)
    
    # Request market data
    tickers = await ib.reqTickersAsync(*contracts)
    
    # Create DataFrame
    data = []
    for ticker in tickers:
        contract = ticker.contract
        expiration = datetime.strptime(contract.lastTradeDateOrContractMonth, '%Y%m%d').date()
        dte = (expiration - today).days
        data.append({
            'Symbol': symbol,
            'StockPrice': stock_price,
            'Strike': contract.strike,
            'Expiration': expiration,
            'DTE': dte,
            'Bid': ticker.bid,
            'BidSize': ticker.bidSize,
            'Ask': ticker.ask,
            'AskSize': ticker.askSize,
            'Contract': contract})
    
    df = pd.DataFrame(data)
    df = df.sort_values(['Expiration', 'Strike'])
    if unfiltered:
        return df
    options_df = df.copy()
    options_df['option_premium'] = options_df['Bid'] / options_df['Strike']
    options_df['annualized_premium'] = options_df['option_premium'] * (365 / options_df['DTE'])
    options_df['safety_margin'] = stock_price - options_df['Strike']
    options_df = options_df[(options_df['option_premium'] > 0.01) & (options_df['annualized_premium'] > 0.1)]
    options_df['safety_margin%'] = options_df['safety_margin'] / options_df['StockPrice']
    
    # Sort the DataFrame after creating all columns
    options_df = options_df.sort_values('safety_margin%', ascending=False)
    
    return options_df.head(10), stock_price

async def get_filtered_put_options(ib, symbols,min_dte = 0,max_dte = 60, unfiltered=False,min_safety_margin=0.05,max_safety_margin=0):
    if not isinstance(symbols, list):
        symbols = [symbols]
        
    tasks = [process_get_filtered_put_options(ib, symbol, min_dte, max_dte, unfiltered,min_safety_margin,max_safety_margin) for symbol in symbols]
    results = await asyncio.gather(*tasks)

    all_options = []
    if not unfiltered:
        for options_df, _ in results:
            if options_df is not None and not options_df.empty:
                all_options.append(options_df)
    else:
        all_options = results
    if all_options:
        return pd.concat(all_options, ignore_index=True)
    else:
        return pd.DataFrame()

In [42]:
df = await get_filtered_put_options(ib, symbols='SPY',unfiltered=True,min_dte=10,min_safety_margin=0.045,max_safety_margin=0.55)
df

Unknown contract: Option(symbol='SPY', lastTradeDateOrContractMonth='20241014', strike=547.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='SPY', lastTradeDateOrContractMonth='20241014', strike=547.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='SPY', lastTradeDateOrContractMonth='20241014', strike=548.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='SPY', lastTradeDateOrContractMonth='20241014', strike=549.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='SPY', lastTradeDateOrContractMonth='20241014', strike=551.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='SPY', lastTradeDateOrContractMonth='20241014', strike=552.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='SPY', lastTradeDateOrContractMonth='20241014', strike=552.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='SPY', lastTradeDateOrContractMonth='20241014', strike=553.0, right='P', exchange='SMART')
Unknown contract

,Symbol,StockPrice,Strike,Expiration,DTE,Bid,BidSize,Ask,AskSize,Contract
0,SPY,572.485,550.0,2024-10-14,10,0.71,1970.0,0.72,246.0,"Option(conId=732657656, symbol='SPY', lastTrad..."
1,SPY,572.485,555.0,2024-10-14,10,1.02,115.0,1.03,871.0,"Option(conId=732826599, symbol='SPY', lastTrad..."
2,SPY,572.485,560.0,2024-10-14,10,1.50,198.0,1.52,1040.0,"Option(conId=732657672, symbol='SPY', lastTrad..."
3,SPY,572.485,563.0,2024-10-14,10,1.91,477.0,1.93,263.0,"Option(conId=732657701, symbol='SPY', lastTrad..."
4,SPY,572.485,564.0,2024-10-14,10,2.08,43.0,2.10,556.0,"Option(conId=732657731, symbol='SPY', lastTrad..."
...,...,...,...,...,...,...,...,...,...,...
649,SPY,572.485,645.0,2024-11-29,56,71.59,15.0,72.52,15.0,"Option(conId=709102543, symbol='SPY', lastTrad..."
650,SPY,572.485,650.0,2024-11-29,56,76.58,15.0,77.52,15.0,"Option(conId=715990400, symbol='SPY', lastTrad..."
651,SPY,572.485,655.0,2024-11-29,56,81.58,15.0,82.52,15.0,"Option(conId=715990427, symbol='SPY', lastTrad..."
652,SPY,572.485,660.0,2024-11-29,56,86.58,15.0,87.52,15.0,"Option(conId=715990460, symbol='SPY', lastTrad..."


In [43]:
df['premium'] = df['Ask'] / df['Strike']
df['annualized_premium'] = df['premium'] * (365 / df['DTE'])
df


,Symbol,StockPrice,Strike,Expiration,DTE,Bid,BidSize,Ask,AskSize,Contract,premium,annualized_premium
0,SPY,572.485,550.0,2024-10-14,10,0.71,1970.0,0.72,246.0,"Option(conId=732657656, symbol='SPY', lastTrad...",0.001309,0.047782
1,SPY,572.485,555.0,2024-10-14,10,1.02,115.0,1.03,871.0,"Option(conId=732826599, symbol='SPY', lastTrad...",0.001856,0.067739
2,SPY,572.485,560.0,2024-10-14,10,1.50,198.0,1.52,1040.0,"Option(conId=732657672, symbol='SPY', lastTrad...",0.002714,0.099071
3,SPY,572.485,563.0,2024-10-14,10,1.91,477.0,1.93,263.0,"Option(conId=732657701, symbol='SPY', lastTrad...",0.003428,0.125124
4,SPY,572.485,564.0,2024-10-14,10,2.08,43.0,2.10,556.0,"Option(conId=732657731, symbol='SPY', lastTrad...",0.003723,0.135904
...,...,...,...,...,...,...,...,...,...,...,...,...
649,SPY,572.485,645.0,2024-11-29,56,71.59,15.0,72.52,15.0,"Option(conId=709102543, symbol='SPY', lastTrad...",0.112434,0.732829
650,SPY,572.485,650.0,2024-11-29,56,76.58,15.0,77.52,15.0,"Option(conId=715990400, symbol='SPY', lastTrad...",0.119262,0.777330
651,SPY,572.485,655.0,2024-11-29,56,81.58,15.0,82.52,15.0,"Option(conId=715990427, symbol='SPY', lastTrad...",0.125985,0.821150
652,SPY,572.485,660.0,2024-11-29,56,86.58,15.0,87.52,15.0,"Option(conId=715990460, symbol='SPY', lastTrad...",0.132606,0.864307


In [44]:
order = Order(df.iloc[0].Contract)

Option(conId=732657656, symbol='SPY', lastTradeDateOrContractMonth='20241014', strike=550.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   241014P00550000', tradingClass='SPY')

In [228]:
symbols

['LW', 'CALM', 'RPM', 'NKE', 'CAG']

In [237]:
import asyncio

In [238]:
async def get_filtered_put_options_async(ib, symbol, max_dte=60, safety_margin=0.5):
    # Get the stock contract
    stock = Stock(symbol, 'SMART', 'USD')
    await ib.qualifyContractsAsync(stock)
    
    # Get the stock price
    [ticker] = await ib.reqTickersAsync(stock)
    stock_price = ticker.marketPrice()
    
    if np.isnan(stock_price):
        print(f"Unable to get market price for {symbol}")
        return None, None
    
    # Calculate strike range
    lower_strike = stock_price * (1-safety_margin)  # 15% below
    upper_strike = stock_price * (1+safety_margin)  # 5% above
    
    # Get option chains
    chains = await ib.reqSecDefOptParamsAsync(stock.symbol, '', stock.secType, stock.conId)
    
    # Get the chain with exchange 'SMART'
    chain = next((c for c in chains if c.exchange == 'SMART'), None)
    if not chain:
        print(f"No SMART chain found for {symbol}")
        return None, None
    
    # Get current date
    today = datetime.now().date()
    
    # Filter expirations and strikes
    valid_expirations = [exp for exp in chain.expirations 
                         if (datetime.strptime(exp, '%Y%m%d').date() - today).days <= max_dte]
    valid_strikes = [strike for strike in chain.strikes 
                     if lower_strike <= strike <= upper_strike]
    
    # Create option contracts
    contracts = [Option(symbol, exp, strike, 'P', 'SMART') 
                 for exp in valid_expirations 
                 for strike in valid_strikes]
    
    # Qualify the contracts
    contracts = await ib.qualifyContractsAsync(*contracts)
    
    # Request market data
    tickers = await ib.reqTickersAsync(*contracts)
    
    # Create DataFrame
    data = []
    for ticker in tickers:
        contract = ticker.contract
        expiration = datetime.strptime(contract.lastTradeDateOrContractMonth, '%Y%m%d').date()
        dte = (expiration - today).days
        data.append({
            'Symbol': symbol,
            'Strike': contract.strike,
            'Expiration': expiration,
            'DTE': dte,
            'Bid': ticker.bid,
            'BidSize': ticker.bidSize,
            'Ask': ticker.ask,
            'AskSize': ticker.askSize,
            'Contract': contract
        })
    
    df = pd.DataFrame(data)
    df = df.sort_values(['Expiration', 'Strike'])
    
    options_df = df.copy()
    options_df['option_premium'] = options_df['Bid'] / options_df['Strike']
    options_df['annualized_premium'] = options_df['option_premium'] * (365 / options_df['DTE'])
    options_df['stock_price'] = stock_price
    options_df['safety_margin'] = stock_price - options_df['Strike']
    options_df = options_df[(options_df['option_premium'] > 0.01) & (options_df['annualized_premium'] > 0.1)]
    options_df['safety_margin%'] = options_df['safety_margin'] / options_df['stock_price']
    
    # Sort the DataFrame after creating all columns
    options_df = options_df.sort_values('safety_margin%', ascending=False)
    
    return options_df.head(10), stock_price

async def process_all_symbols(ib, symbols):
    tasks = [get_filtered_put_options_async(ib, symbol) for symbol in symbols]
    results = await asyncio.gather(*tasks)
    
    all_options = []
    for options_df, stock_price in results:
        if options_df is not None and not options_df.empty:
            all_options.append(options_df)
    
    if all_options:
        return pd.concat(all_options, ignore_index=True)
    else:
        return pd.DataFrame()

In [239]:
symbols = ['LW', 'CALM', 'RPM', 'NKE', 'CAG']  # Your list of symbols
result_df = await process_all_symbols(ib, symbols)
    
if not result_df.empty:
    result_df.sort_values('safety_margin%', ascending=False)

Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241004', strike=72.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241004', strike=77.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241004', strike=82.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241011', strike=72.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241011', strike=77.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241011', strike=82.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241025', strike=72.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241025', strike=77.5, right='P', exchange='SMART')
Unknown contract: Option

In [10]:
result_df['decision_ind'] = result_df['safety_margin%'] / result_df['option_premium']
result_df.sort_values('decision_ind', ascending=False)

NameError: name 'result_df' is not defined

In [229]:
options_df['safety_margin%'] = options_df['safety_margin'] / options_df['stock_price']
options_df


,Strike,Expiration,DTE,Bid,BidSize,Ask,AskSize,Contract,option_premium,annualized_premium,stock_price,safety_margin,safety_margin%
97,77.5,2024-11-15,44,0.96,790.0,1.06,2459.0,"Option(conId=719526311, symbol='NKE', lastTrad...",0.012387,0.102757,82.555,5.055,0.061232
87,79.0,2024-11-08,37,0.88,339.0,1.29,137.0,"Option(conId=731946413, symbol='NKE', lastTrad...",0.011139,0.109887,82.555,3.555,0.043062
88,80.0,2024-11-08,37,1.41,167.0,1.56,99.0,"Option(conId=731946473, symbol='NKE', lastTrad...",0.017625,0.173868,82.555,2.555,0.030949
98,80.0,2024-11-15,44,1.66,36.0,1.72,899.0,"Option(conId=705993053, symbol='NKE', lastTrad...",0.020750,0.172131,82.555,2.555,0.030949
60,80.0,2024-10-25,23,0.93,1040.0,1.04,2268.0,"Option(conId=727047636, symbol='NKE', lastTrad...",0.011625,0.184484,82.555,2.555,0.030949
76,80.0,2024-11-01,30,1.10,139.0,1.19,518.0,"Option(conId=728253068, symbol='NKE', lastTrad...",0.013750,0.167292,82.555,2.555,0.030949
77,81.0,2024-11-01,30,1.40,977.0,1.57,1037.0,"Option(conId=728253102, symbol='NKE', lastTrad...",0.017284,0.210288,82.555,1.555,0.018836
61,81.0,2024-10-25,23,1.24,869.0,1.43,4500.0,"Option(conId=727047673, symbol='NKE', lastTrad...",0.015309,0.242941,82.555,1.555,0.018836
44,81.0,2024-10-18,16,1.04,155.0,1.12,1978.0,"Option(conId=730757679, symbol='NKE', lastTrad...",0.012840,0.292901,82.555,1.555,0.018836
89,81.0,2024-11-08,37,1.72,115.0,1.95,175.0,"Option(conId=731946531, symbol='NKE', lastTrad...",0.021235,0.209476,82.555,1.555,0.018836


In [220]:
a, _ = get_filtered_put_options(ib, symbols[0])

In [225]:
a

,Strike,Expiration,DTE,Bid,BidSize,Ask,AskSize,Contract,option_premium,annualized_premium,stock_price,safety_margin
6,60.0,2024-11-15,44,0.95,177.0,1.15,713.0,"Option(conId=729997681, symbol='LW', lastTrade...",0.015833,0.131345,65.67,5.67
2,62.5,2024-10-18,16,0.80,11.0,0.95,551.0,"Option(conId=718404917, symbol='LW', lastTrade...",0.012800,0.292000,65.67,3.17
7,62.5,2024-11-15,44,1.60,39.0,1.95,179.0,"Option(conId=729997734, symbol='LW', lastTrade...",0.025600,0.212364,65.67,3.17
3,65.0,2024-10-18,16,1.65,389.0,1.90,10.0,"Option(conId=684662461, symbol='LW', lastTrade...",0.025385,0.579087,65.67,0.67
8,65.0,2024-11-15,44,2.60,3.0,2.95,1112.0,"Option(conId=729997787, symbol='LW', lastTrade...",0.040000,0.331818,65.67,0.67
4,67.5,2024-10-18,16,2.90,78.0,3.30,18.0,"Option(conId=721751619, symbol='LW', lastTrade...",0.042963,0.980093,65.67,-1.83
9,67.5,2024-11-15,44,3.80,32.0,4.30,3.0,"Option(conId=729997854, symbol='LW', lastTrade...",0.056296,0.467003,65.67,-1.83


In [221]:
b, _ = get_filtered_put_options(ib, symbols[1])

In [226]:
b

,Strike,Expiration,DTE,Bid,BidSize,Ask,AskSize,Contract,option_premium,annualized_premium,stock_price,safety_margin
7,70.0,2024-11-15,44,0.90,305.0,1.25,117.0,"Option(conId=690655193, symbol='CALM', lastTra...",0.012857,0.106656,76.6,6.6
8,72.5,2024-11-15,44,1.50,369.0,2.05,166.0,"Option(conId=695628006, symbol='CALM', lastTra...",0.020690,0.171630,76.6,4.1
3,75.0,2024-10-18,16,1.00,528.0,1.35,356.0,"Option(conId=722829516, symbol='CALM', lastTra...",0.013333,0.304167,76.6,1.6
9,75.0,2024-11-15,44,2.45,202.0,2.90,104.0,"Option(conId=690655266, symbol='CALM', lastTra...",0.032667,0.270985,76.6,1.6
4,77.5,2024-10-18,16,2.15,120.0,2.50,209.0,"Option(conId=722829606, symbol='CALM', lastTra...",0.027742,0.632863,76.6,-0.9
10,77.5,2024-11-15,44,3.50,191.0,4.20,127.0,"Option(conId=719488007, symbol='CALM', lastTra...",0.045161,0.374633,76.6,-0.9
5,80.0,2024-10-18,16,3.40,393.0,4.30,203.0,"Option(conId=722829683, symbol='CALM', lastTra...",0.042500,0.969531,76.6,-3.4
11,80.0,2024-11-15,44,5.10,99.0,5.70,160.0,"Option(conId=690655368, symbol='CALM', lastTra...",0.063750,0.528835,76.6,-3.4


In [222]:
c, _ = get_filtered_put_options(ib, symbols[2])

In [227]:
c

,Strike,Expiration,DTE,Bid,BidSize,Ask,AskSize,Contract,option_premium,annualized_premium,stock_price,safety_margin
7,120.0,2024-11-15,44,2.00,203.0,2.35,73.0,"Option(conId=690977232, symbol='RPM', lastTrad...",0.016667,0.138258,124.52,4.52
3,125.0,2024-10-18,16,2.45,169.0,2.95,140.0,"Option(conId=723235348, symbol='RPM', lastTrad...",0.019600,0.447125,124.52,-0.48
8,125.0,2024-11-15,44,3.80,314.0,4.30,65.0,"Option(conId=690977306, symbol='RPM', lastTrad...",0.030400,0.252182,124.52,-0.48
4,130.0,2024-10-18,16,4.70,212.0,8.30,132.0,"Option(conId=723235423, symbol='RPM', lastTrad...",0.036154,0.824760,124.52,-5.48
9,130.0,2024-11-15,44,6.40,589.0,7.40,71.0,"Option(conId=690977383, symbol='RPM', lastTrad...",0.049231,0.408392,124.52,-5.48


In [223]:
d, _ = get_filtered_put_options(ib, symbols[3])
d

Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241004', strike=72.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241004', strike=77.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241004', strike=82.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241011', strike=72.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241011', strike=77.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241011', strike=82.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241025', strike=72.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241025', strike=77.5, right='P', exchange='SMART')
Unknown contract: Option

,Strike,Expiration,DTE,Bid,BidSize,Ask,AskSize,Contract,option_premium,annualized_premium,stock_price,safety_margin
104,77.5,2024-11-15,44,0.98,1995.0,1.05,898.0,"Option(conId=719526311, symbol='NKE', lastTrad...",0.012645,0.104897,82.13,4.63
92,78.0,2024-11-08,37,0.82,430.0,1.00,1.0,"Option(conId=731946347, symbol='NKE', lastTrad...",0.010513,0.103708,82.13,4.13
93,79.0,2024-11-08,37,0.98,356.0,1.29,121.0,"Option(conId=731946413, symbol='NKE', lastTrad...",0.012405,0.122374,82.13,3.13
105,80.0,2024-11-15,44,1.69,132.0,1.74,214.0,"Option(conId=705993053, symbol='NKE', lastTrad...",0.021125,0.175241,82.13,2.13
64,80.0,2024-10-25,23,0.92,3911.0,1.05,35.0,"Option(conId=727047636, symbol='NKE', lastTrad...",0.011500,0.182500,82.13,2.13
94,80.0,2024-11-08,37,1.35,388.0,1.74,952.0,"Option(conId=731946473, symbol='NKE', lastTrad...",0.016875,0.166470,82.13,2.13
81,80.0,2024-11-01,30,1.12,506.0,1.22,749.0,"Option(conId=728253068, symbol='NKE', lastTrad...",0.014000,0.170333,82.13,2.13
82,81.0,2024-11-01,30,1.46,22.0,1.99,2167.0,"Option(conId=728253102, symbol='NKE', lastTrad...",0.018025,0.219300,82.13,1.13
65,81.0,2024-10-25,23,1.30,1.0,1.49,3228.0,"Option(conId=727047673, symbol='NKE', lastTrad...",0.016049,0.254697,82.13,1.13
47,81.0,2024-10-18,16,1.08,569.0,1.30,6167.0,"Option(conId=730757679, symbol='NKE', lastTrad...",0.013333,0.304167,82.13,1.13


In [224]:
e, _ = get_filtered_put_options(ib, symbols[4])
e

Unknown contract: Option(symbol='CAG', lastTradeDateOrContractMonth='20241025', strike=27.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='CAG', lastTradeDateOrContractMonth='20241025', strike=28.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='CAG', lastTradeDateOrContractMonth='20241025', strike=29.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='CAG', lastTradeDateOrContractMonth='20241025', strike=30.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='CAG', lastTradeDateOrContractMonth='20241025', strike=31.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='CAG', lastTradeDateOrContractMonth='20241101', strike=27.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='CAG', lastTradeDateOrContractMonth='20241101', strike=28.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='CAG', lastTradeDateOrContractMonth='20241101', strike=29.5, right='P', exchange='SMART')
Unknown contract: Option

,Strike,Expiration,DTE,Bid,BidSize,Ask,AskSize,Contract,option_premium,annualized_premium,stock_price,safety_margin
53,28.0,2024-11-15,44,0.30,3752.0,0.40,1898.0,"Option(conId=729765398, symbol='CAG', lastTrad...",0.010714,0.088880,30.13,2.13
42,29.0,2024-11-01,30,0.35,2776.0,0.55,487.0,"Option(conId=728192123, symbol='CAG', lastTrad...",0.012069,0.146839,30.13,1.13
54,29.0,2024-11-15,44,0.55,2480.0,0.65,1499.0,"Option(conId=729765446, symbol='CAG', lastTrad...",0.018966,0.157328,30.13,1.13
48,29.0,2024-11-08,37,0.45,422.0,0.65,67.0,"Option(conId=731851019, symbol='CAG', lastTrad...",0.015517,0.153075,30.13,1.13
28,29.5,2024-10-18,16,0.30,2129.0,0.40,628.0,"Option(conId=730690510, symbol='CAG', lastTrad...",0.010169,0.231992,30.13,0.63
37,30.0,2024-10-25,23,0.50,3474.0,0.65,888.0,"Option(conId=727000154, symbol='CAG', lastTrad...",0.016667,0.264493,30.13,0.13
55,30.0,2024-11-15,44,0.95,1541.0,1.05,1641.0,"Option(conId=729765493, symbol='CAG', lastTrad...",0.031667,0.262689,30.13,0.13
18,30.0,2024-10-11,9,0.40,1520.0,0.50,730.0,"Option(conId=725847510, symbol='CAG', lastTrad...",0.013333,0.540741,30.13,0.13
49,30.0,2024-11-08,37,0.80,417.0,1.05,68.0,"Option(conId=731851069, symbol='CAG', lastTrad...",0.026667,0.263063,30.13,0.13
29,30.0,2024-10-18,16,0.50,1267.0,0.60,3711.0,"Option(conId=722834139, symbol='CAG', lastTrad...",0.016667,0.380208,30.13,0.13


In [200]:
options_df['option_premium'] = options_df['Bid'] / options_df['Strike']
options_df['annualized_premium'] = options_df['option_premium'] * (365 / options_df['DTE'])
options_df['stock_price'] = stock_price
options_df['safety_margin'] = stock_price - options_df['Strike']


